In [2]:
import os
%load_ext rpy2.ipython
#%load_ext pushnote

In [3]:
workDir = '/home/chantal/Chazy/Chazy_SIP/data/'
figDir = '/home/chantal/Chazy/Chazy_SIP/data/figs/'

l2fcResFile = os.path.join(workDir, 'l2fc_sparse-maxRejH.txt')

physeqDir = '/home/chantal/Chazy/data/phyloseq/'
physeqBulk = 'bulk-core'

In [4]:
%%R
library(phyloseq)
library(dplyr)
library(tidyr)
library(ggplot2)
library(grid)
library(gridExtra)
library(foreach)
library(doParallel)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘gridExtra’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following object is masked from ‘package:dplyr’:

    combine


  res 

In [5]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)
%cd $workDir

if not os.path.isdir(figDir):
    os.makedirs(figDir)

/home/chantal/Chazy/Chazy_SIP/data


In [7]:
%%R -i workDir -i figDir -i physeqDir -i physeqBulk  -i l2fcResFile 
cat('variables loaded into R\n')

variables loaded into R


In [10]:
%%R 

F = file.path(physeqDir, physeqBulk)
physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
physeq.Bulk = prune_samples(physeq.Bulk.m$Sample_Date == '9/24/14',
                           physeq.Bulk)
physeq.Bulk

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 21374 taxa and 16 samples ]
sample_data() Sample Data:       [ 16 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 21374 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 21374 tips and 21373 internal nodes ]


In [11]:
%%R
df.l2fc = read.delim(l2fcResFile, sep='\t')
df.l2fc %>% head(n=3)

  Treatment Day Replicate Land_Management      OTU log2FoldChange         p
1       13C   3         4             NTH OTU.4163      0.2905525 0.4758928
2       13C   3         4             NTH OTU.5493     -0.2477539 0.8663980
3       13C   3         4             NTH OTU.1739     -0.3623365 0.9103254
  padj    Rank1            Rank2   Rank3                  Rank4 Rank5 Rank6
1    1 Bacteria    __Chloroflexi  __S085 __uncultured_bacterium  <NA>  <NA>
2    1 Bacteria    __Chloroflexi  __S085 __uncultured_bacterium  <NA>  <NA>
3    1 Bacteria __Planctomycetes __OM190 __uncultured_bacterium  <NA>  <NA>
  Rank7 Rank8 density_min density_max sparsity_threshold
1  <NA>  <NA>         1.7        1.73               0.05
2  <NA>  <NA>         1.7        1.73               0.05
3  <NA>  <NA>         1.7        1.73               0.05


In [46]:
%%R
df.13C.N = filter(df.l2fc, padj <=0.10, Treatment == '13C', Land_Management == 'NTH')
df.13C.P = filter(df.l2fc, padj <=0.10, Treatment == '13C', Land_Management == 'PTH')
df.13X.N = filter(df.l2fc, padj <=0.10, Treatment == '13X', Land_Management == 'NTH')
df.13X.P = filter(df.l2fc, padj <=0.10, Treatment == '13X', Land_Management == 'PTH')

In [111]:
%%R
physeq.Bulk.m = physeq.Bulk %>% sample_data

physeq.NTH = prune_samples(physeq.Bulk.m$Land_Management == 'NTH',
                           physeq.Bulk)

physeq.PTH = prune_samples(physeq.Bulk.m$Land_Management == 'PTH',
                           physeq.Bulk)

df.13C.N.phyl = prune_taxa(as.character(df.13C.N$OTU),
                           physeq.NTH) 

df.13C.P.phyl = prune_taxa(as.character(df.13C.P$OTU),
                           physeq.PTH) 

df.13X.N.phyl = prune_taxa(as.character(df.13X.N$OTU),
                           physeq.NTH) 

df.13X.P.phyl = prune_taxa(as.character(df.13X.P$OTU),
                           physeq.PTH) 

In [112]:
%%R
print(df.13C.N.phyl)
print(length(unique(df.13C.N$OTU)))

print(df.13C.P.phyl)
print(length(unique(df.13C.P$OTU)))

print(df.13X.N.phyl)
print(length(unique(df.13X.N$OTU)))

print(df.13X.P.phyl)
print(length(unique(df.13X.P$OTU)))

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1031 taxa and 4 samples ]
sample_data() Sample Data:       [ 4 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 1031 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1031 tips and 1030 internal nodes ]
[1] 1062
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 918 taxa and 4 samples ]
sample_data() Sample Data:       [ 4 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 918 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 918 tips and 917 internal nodes ]
[1] 935
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 567 taxa and 4 samples ]
sample_data() Sample Data:       [ 4 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 567 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 567 tips and 566 internal nodes ]
[1] 582
phyloseq-class experiment-level object
otu_table()   OTU

## Looking at general alpha diversity of responders, time summed together

In [115]:
%%R
library(picante)

otu_tab.13CN = t(otu_table(df.13C.N.phyl)) %>% as.data.frame
tree.13CN = (phy_tree(df.13C.N.phyl))

otu_tab.13CP = t(otu_table(df.13C.P.phyl)) %>% as.data.frame
tree.13CP = (phy_tree(df.13C.P.phyl))

otu_tab.13XN = t(otu_table(df.13X.N.phyl)) %>% as.data.frame
tree.13XN = (phy_tree(df.13X.N.phyl))

otu_tab.13XP = t(otu_table(df.13X.P.phyl)) %>% as.data.frame
tree.13XP = (phy_tree(df.13X.P.phyl))



In [268]:
%%R
pd.13CN = pd(otu_tab.13CN, tree.13CN)
pd.13CP = pd(otu_tab.13CP, tree.13CP)
pd.13XN = pd(otu_tab.13XN, tree.13XN)
pd.13XP = pd(otu_tab.13XP, tree.13XP)

In [269]:
%%R
pd.13CN$Sample = rownames(pd.13CN)
pd.13CN = pd.13CN %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13CN.arr = arrange(pd.13CN, Replicate)
pd.13CN.arr

pd.13CN %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management mean_PD    sd_PD
            (chr)   (dbl)    (dbl)
1             NTH 47.5755 4.559555


In [270]:
%%R
pd.13CP$Sample = rownames(pd.13CP)
pd.13CP = pd.13CP %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13CP.arr = arrange(pd.13CP, Replicate)
pd.13CP.arr

pd.13CP %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD    sd_PD
            (chr)    (dbl)    (dbl)
1             PTH 51.44008 4.159203


In [231]:
%%R
t.test(pd.13CN.arr$PD, pd.13CP.arr$PD, paired = TRUE)


	Paired t-test

data:  pd.13CN.arr$PD and pd.13CP.arr$PD
t = -1.2704, df = 3, p-value = 0.2935
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -13.546002   5.816832
sample estimates:
mean of the differences 
              -3.864585 



In [271]:
%%R
pd.13XN$Sample = rownames(pd.13XN)
pd.13XN = pd.13XN %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13XN.arr = arrange(pd.13XN, Replicate)
pd.13XN.arr

pd.13XN %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD    sd_PD
            (chr)    (dbl)    (dbl)
1             NTH 26.40475 3.005067


In [272]:
%%R
pd.13XP$Sample = rownames(pd.13XP)
pd.13XP = pd.13XP %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13XP.arr = arrange(pd.13XP, Replicate)
pd.13XP.arr

pd.13XP %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD    sd_PD
            (chr)    (dbl)    (dbl)
1             PTH 30.32662 2.125522


In [232]:
%%R
t.test(pd.13XN.arr$PD, pd.13XP.arr$PD, paired = TRUE)


	Paired t-test

data:  pd.13XN.arr$PD and pd.13XP.arr$PD
t = -2.4167, df = 3, p-value = 0.09445
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -9.086354  1.242614
sample estimates:
mean of the differences 
               -3.92187 



## Pielou's for Incorporator

In [274]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13CN <- diversity(otu_tab.13CN )
## Species richness (S) and Pielou's evenness (J):
S.13CN <- specnumber(otu_tab.13CN ) ## rowSums(BCI > 0) does the same...
J.13CN <- H.13CN/log(S.13CN)

J.13CN <- J.13CN[c(1,3,4,2)]

print(mean(J.13CN))
print(sd(J.13CN))

[1] 0.8275907
[1] 0.04194244


In [275]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness
H.13CP <- diversity(otu_tab.13CP)
## Species richness (S) and Pielou's evenness (J):
S.13CP <- specnumber(otu_tab.13CP) ## rowSums(BCI > 0) does the same...
J.13CP <- H.13CP/log(S.13CP)

J.13CP <- J.13CP[c(4,3,1,2)]

J.13CP

print(mean(J.13CP))
print(sd(J.13CP))

[1] 0.8487474
[1] 0.01738892


In [185]:
%%R
t.test(J.13CN, J.13CP, paired = TRUE)


	Paired t-test

data:  J.13CN and J.13CP
t = -0.89958, df = 3, p-value = 0.4346
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.09600249  0.05368907
sample estimates:
mean of the differences 
            -0.02115671 



In [276]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13XN <- diversity(otu_tab.13XN )
## Species richness (S) and Pielou's evenness (J):
S.13XN <- specnumber(otu_tab.13XN ) ## rowSums(BCI > 0) does the same...
J.13XN <- H.13XN/log(S.13XN)

J.13XN <- J.13XN[c(1,3,4,2)]

J.13XN
print(mean(J.13XN))
print(sd(J.13XN))

[1] 0.8407728
[1] 0.01239185


In [277]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13XP <- diversity(otu_tab.13XP )
## Species richness (S) and Pielou's evenness (J):
S.13XP <- specnumber(otu_tab.13XP ) ## rowSums(BCI > 0) does the same...
J.13XP <- H.13XP/log(S.13XP)

J.13XP <- J.13XP[c(4,3,1,2)]

J.13XP
print(mean(J.13XP))
print(sd(J.13XP))

[1] 0.8175502
[1] 0.02106183


In [189]:
%%R
t.test(J.13XN, J.13XP, Paired = TRUE)


	Welch Two Sample t-test

data:  J.13XN and J.13XP
t = 1.9006, df = 4.8547, p-value = 0.1175
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.008470631  0.054915860
sample estimates:
mean of x mean of y 
0.8407728 0.8175502 



## Shannon for Incorporator

In [208]:
%%R
rich.13CN = estimate_richness(df.13C.N.phyl)

rich.13CP = estimate_richness(df.13C.P.phyl)

rich.13XN = estimate_richness(df.13X.N.phyl)

rich.13XP = estimate_richness(df.13X.P.phyl)

In [209]:
%%R
rich.13CN$Sample = rownames(rich.13CN)

rich.13CN = rich.13CN  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13CN %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             NTH         496.5   606.2063      24.27431 609.7073    11.72881
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     5.133124    0.9716756        60.34067    158.2118         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd  ACE_sd se.ACE_sd Shannon_sd
1             NA    65.67851 72.87167     5.20564 69.9599 0.8105175  0.3079686
  Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.02113421      44.88448  18.72886       NA     1.290994


In [210]:
%%R
rich.13CP$Sample = rownames(rich.13CP)

rich.13CP = rich.13CP  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13CP %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             PTH           529   613.6528      19.65626 620.8731    11.59847
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     5.319512    0.9886425        89.77027    154.4631         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA     45.4386 53.31854     4.89746 57.76424 0.7917428 0.08296392
   Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.001914824      13.68096  14.40343       NA     1.290994


In [211]:
%%R
rich.13CN.arr = rich.13CN %>% arrange(Replicate)
rich.13CP.arr = rich.13CP %>% arrange(Replicate)

In [212]:
%%R
head(rich.13CP.arr)

  Observed    Chao1 se.chao1      ACE   se.ACE  Shannon   Simpson InvSimpson
1      505 555.5096 13.22321 570.4660 10.75474 5.374482 0.9901996  102.03696
2      490 608.3093 25.13767 617.2611 11.81397 5.347208 0.9897431   97.49580
3      528 606.0000 19.94172 593.1232 11.23255 5.196193 0.9859295   71.07053
4      593 684.7925 20.32245 702.6420 12.59263 5.360164 0.9886977   88.47779
    Fisher Month Land_Management Replicate
1 150.8927     S             PTH         1
2 166.5072     S             PTH         2
3 135.5860     S             PTH         3
4 164.8663     S             PTH         4


In [213]:
%%R
t.test(rich.13CN.arr$Observed, rich.13CP.arr$Observed, paired = TRUE)


	Paired t-test

data:  rich.13CN.arr$Observed and rich.13CP.arr$Observed
t = -0.88233, df = 3, p-value = 0.4426
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -149.72251   84.72251
sample estimates:
mean of the differences 
                  -32.5 



In [214]:
%%R
t.test(rich.13CN.arr$Shannon, rich.13CP.arr$Shannon, paired = TRUE)


	Paired t-test

data:  rich.13CN.arr$Shannon and rich.13CP.arr$Shannon
t = -1.0951, df = 3, p-value = 0.3535
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.7280436  0.3552688
sample estimates:
mean of the differences 
             -0.1863874 



In [215]:
%%R
t.test(rich.13CN.arr$Chao1, rich.13CP.arr$Chao1, paired = TRUE)


	Paired t-test

data:  rich.13CN.arr$Chao1 and rich.13CP.arr$Chao1
t = -0.15782, df = 3, p-value = 0.8846
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -157.6097  142.7166
sample estimates:
mean of the differences 
              -7.446558 



In [216]:
%%R
t.test(rich.13CN.arr$Simpson, rich.13CP.arr$Simpson, paired = TRUE)


	Paired t-test

data:  rich.13CN.arr$Simpson and rich.13CP.arr$Simpson
t = -1.5526, df = 3, p-value = 0.2183
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.05174421  0.01781045
sample estimates:
mean of the differences 
            -0.01696688 



In [217]:
%%R
rich.13XN$Sample = rownames(rich.13XN)

rich.13XN = rich.13XN  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13XN %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             NTH           306   374.5269       20.2238 368.4516    9.241235
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     4.805424     0.983057        59.23693    87.63207         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    42.70051 44.04882    4.892434 41.89057  0.546499 0.06945852
   Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.001189573      4.098264  9.285781       NA     1.290994


In [218]:
%%R
rich.13XP$Sample = rownames(rich.13XP)

rich.13XP = rich.13XP  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13XP %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             PTH        323.75   387.5382      18.11292 388.2508    9.179157
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     4.722524    0.9765464        43.82602    96.98718         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    28.76775 11.33187    6.837812 15.63472 0.4516062  0.1174084
   Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.004888301      7.610676  7.677888       NA     1.290994


In [219]:
%%R
rich.13XN.arr = rich.13XN %>% arrange(Replicate)
rich.13XP.arr = rich.13XP %>% arrange(Replicate)

In [220]:
%%R
t.test(rich.13XN.arr$Observed, rich.13XP.arr$Observed, paired = TRUE)


	Paired t-test

data:  rich.13XN.arr$Observed and rich.13XP.arr$Observed
t = -0.723, df = 3, p-value = 0.522
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -95.88079  60.38079
sample estimates:
mean of the differences 
                 -17.75 



In [221]:
%%R
t.test(rich.13XN.arr$Shannon, rich.13XP.arr$Shannon, paired = TRUE)


	Paired t-test

data:  rich.13XN.arr$Shannon and rich.13XP.arr$Shannon
t = 2.2091, df = 3, p-value = 0.1142
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03652556  0.20232574
sample estimates:
mean of the differences 
             0.08290009 



In [222]:
%%R
t.test(rich.13XN.arr$Chao1, rich.13XP.arr$Chao1, paired = TRUE)


	Paired t-test

data:  rich.13XN.arr$Chao1 and rich.13XP.arr$Chao1
t = -0.55943, df = 3, p-value = 0.6149
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -87.02919  61.00645
sample estimates:
mean of the differences 
              -13.01137 



In [223]:
%%R
t.test(rich.13XN.arr$Simpson, rich.13XP.arr$Simpson, paired = TRUE)


	Paired t-test

data:  rich.13XN.arr$Simpson and rich.13XP.arr$Simpson
t = 3.2769, df = 3, p-value = 0.04653
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.0001877238 0.0128334551
sample estimates:
mean of the differences 
            0.006510589 



## Alpha diversity for entire community, not just incorporators (PTH vs NTH)

In [233]:
%%R
physeq.NTH = prune_samples(physeq.Bulk.m$Land_Management == 'NTH',
                           physeq.Bulk)

physeq.PTH = prune_samples(physeq.Bulk.m$Land_Management == 'PTH',
                           physeq.Bulk)

In [234]:
%%R
otu_tab.NTH = t(otu_table(physeq.NTH)) %>% as.data.frame
tree.NTH = (phy_tree(physeq.NTH))

otu_tab.PTH = t(otu_table(physeq.PTH)) %>% as.data.frame
tree.PTH = (phy_tree(physeq.PTH))

In [279]:
%%R
pd.NTH = pd(otu_tab.NTH, tree.NTH)
pd.PTH = pd(otu_tab.PTH, tree.PTH)

In [280]:
%%R
pd.NTH$Sample = rownames(pd.NTH)
pd.NTH = pd.NTH %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.NTH.arr = arrange(pd.NTH, Replicate)
pd.NTH.arr

pd.NTH %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD   sd_PD
            (chr)    (dbl)   (dbl)
1             NTH 222.3612 9.02947


In [281]:
%%R
pd.PTH$Sample = rownames(pd.PTH)
pd.PTH = pd.PTH %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.PTH.arr = arrange(pd.PTH, Replicate)

pd.PTH %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD    sd_PD
            (chr)    (dbl)    (dbl)
1             PTH 224.4609 30.62547


In [238]:
%%R
t.test(pd.NTH.arr$PD, pd.PTH.arr$PD, paired = TRUE)


	Paired t-test

data:  pd.NTH.arr$PD and pd.PTH.arr$PD
t = -0.14946, df = 3, p-value = 0.8907
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -46.80881  42.60931
sample estimates:
mean of the differences 
              -2.099753 



### Pielou's - whole community

In [282]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.NTH <- diversity(otu_tab.NTH )
## Species richness (S) and Pielou's evenness (J):
S.NTH <- specnumber(otu_tab.NTH ) ## rowSums(BCI > 0) does the same...
J.NTH <- H.NTH/log(S.NTH)

J.NTH <- J.NTH[c(1,3,4,2)]

J.NTH
print(mean(J.NTH))
print(sd(J.NTH))

[1] 0.8749483
[1] 0.01217904


In [283]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.PTH <- diversity(otu_tab.PTH )
## Species richness (S) and Pielou's evenness (J):
S.PTH <- specnumber(otu_tab.PTH ) ## rowSums(BCI > 0) does the same...
J.PTH <- H.PTH/log(S.PTH)

J.PTH <- J.PTH[c(4,3,1,2)]

J.PTH

print(mean(J.PTH))
print(sd(J.PTH))

[1] 0.890898
[1] 0.009642798


In [245]:
%%R
t.test(J.NTH, J.PTH, paired = TRUE)


	Paired t-test

data:  J.NTH and J.PTH
t = -2.0284, df = 3, p-value = 0.1356
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.040974051  0.009074686
sample estimates:
mean of the differences 
            -0.01594968 



## Shannon Diversity - whole community

In [258]:
%%R
rich.NTH = estimate_richness(physeq.NTH)

rich.PTH = estimate_richness(physeq.PTH)


In [259]:
%%R
rich.NTH$Sample = rownames(rich.NTH)

rich.NTH = rich.NTH  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.NTH %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             NTH       3977.75   5358.184       95.5188 5509.241    38.59164
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     7.249965    0.9976317        432.0535    1359.122         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    263.6834  552.103    17.70566 656.9174  4.765003 0.04527208
    Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.0004198162      74.55878  115.2496       NA     1.290994


In [260]:
%%R
rich.PTH$Sample = rownames(rich.PTH)

rich.PTH = rich.PTH  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.PTH %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             PTH        4055.5   5491.503      97.27214 5678.876    39.87886
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     7.389195    0.9983093        593.3054    1507.453         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd  ACE_sd se.ACE_sd Shannon_sd
1             NA    716.8424 762.7824    9.048053 707.194   1.72394 0.08585133
    Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.0001089424      37.61282   105.045       NA     1.290994


In [261]:
%%R
rich.NTH.arr = rich.NTH %>% arrange(Replicate)
rich.PTH.arr = rich.PTH %>% arrange(Replicate)

In [263]:
%%R
t.test(rich.NTH.arr$Observed, rich.PTH.arr$Observed, paired = TRUE)


	Paired t-test

data:  rich.NTH.arr$Observed and rich.PTH.arr$Observed
t = -0.22426, df = 3, p-value = 0.837
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1181.106  1025.606
sample estimates:
mean of the differences 
                 -77.75 



In [264]:
%%R
t.test(rich.NTH.arr$Chao1, rich.PTH.arr$Chao1, paired = TRUE)


	Paired t-test

data:  rich.NTH.arr$Chao1 and rich.PTH.arr$Chao1
t = -0.2531, df = 3, p-value = 0.8165
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1809.70  1543.06
sample estimates:
mean of the differences 
              -133.3199 



In [265]:
%%R
t.test(rich.NTH.arr$Shannon, rich.PTH.arr$Shannon, paired = TRUE)


	Paired t-test

data:  rich.NTH.arr$Shannon and rich.PTH.arr$Shannon
t = -2.7058, df = 3, p-value = 0.07342
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.30298676  0.02452658
sample estimates:
mean of the differences 
             -0.1392301 



In [266]:
%%R
t.test(rich.NTH.arr$Simpson, rich.PTH.arr$Simpson, paired = TRUE)


	Paired t-test

data:  rich.NTH.arr$Simpson and rich.PTH.arr$Simpson
t = -2.7939, df = 3, p-value = 0.0682
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.449459e-03  9.422481e-05
sample estimates:
mean of the differences 
          -0.0006776169 



## Looking at Cellulose Day 3 and Xylose D1 Incorporator Communities

In [284]:
%%R
df.13C.N.D3 = filter(df.l2fc, padj <=0.10, Treatment == '13C', Land_Management == 'NTH', Day == 3)
df.13C.P.D3 = filter(df.l2fc, padj <=0.10, Treatment == '13C', Land_Management == 'PTH', Day == 3)
df.13X.N.D1 = filter(df.l2fc, padj <=0.10, Treatment == '13X', Land_Management == 'NTH', Day == 1)
df.13X.P.D1 = filter(df.l2fc, padj <=0.10, Treatment == '13X', Land_Management == 'PTH', Day == 1)

In [285]:
%%R

physeq.Bulk.m = physeq.Bulk %>% sample_data

physeq.NTH = prune_samples(physeq.Bulk.m$Land_Management == 'NTH',
                           physeq.Bulk)

physeq.PTH = prune_samples(physeq.Bulk.m$Land_Management == 'PTH',
                           physeq.Bulk)

df.13C.N.phyl = prune_taxa(as.character(df.13C.N.D3$OTU),
                           physeq.NTH) 

df.13C.P.phyl = prune_taxa(as.character(df.13C.P.D3$OTU),
                           physeq.PTH) 

df.13X.N.phyl = prune_taxa(as.character(df.13X.N.D1$OTU),
                           physeq.NTH) 

df.13X.P.phyl = prune_taxa(as.character(df.13X.P$OTU),
                           physeq.PTH) 

### Faith's PD

In [286]:
%%R
library(picante)

otu_tab.13CN = t(otu_table(df.13C.N.phyl)) %>% as.data.frame
tree.13CN = (phy_tree(df.13C.N.phyl))

otu_tab.13CP = t(otu_table(df.13C.P.phyl)) %>% as.data.frame
tree.13CP = (phy_tree(df.13C.P.phyl))

otu_tab.13XN = t(otu_table(df.13X.N.phyl)) %>% as.data.frame
tree.13XN = (phy_tree(df.13X.N.phyl))

otu_tab.13XP = t(otu_table(df.13X.P.phyl)) %>% as.data.frame
tree.13XP = (phy_tree(df.13X.P.phyl))



In [293]:
%%R
pd.13CN = pd(otu_tab.13CN, tree.13CN)
pd.13CP = pd(otu_tab.13CP, tree.13CP)
pd.13XN = pd(otu_tab.13XN, tree.13XN)
pd.13XP = pd(otu_tab.13XP, tree.13XP)

In [288]:
%%R
pd.13CN$Sample = rownames(pd.13CN)
pd.13CN = pd.13CN %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13CN.arr = arrange(pd.13CN, Replicate)
pd.13CN.arr

pd.13CN %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management mean_PD     sd_PD
            (chr)   (dbl)     (dbl)
1             NTH 1.59699 0.5782433


In [289]:
%%R
pd.13CP$Sample = rownames(pd.13CP)
pd.13CP = pd.13CP %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13CP.arr = arrange(pd.13CP, Replicate)
pd.13CP.arr

pd.13CP %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD   sd_PD
            (chr)    (dbl)   (dbl)
1             PTH 16.66043 1.33431


In [297]:
%%R
t.test(pd.13CN.arr$PD, pd.13CP.arr$PD, paired = TRUE)


	Paired t-test

data:  pd.13CN.arr$PD and pd.13CP.arr$PD
t = -16.124, df = 3, p-value = 0.0005189
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -18.03663 -12.09025
sample estimates:
mean of the differences 
              -15.06344 



In [294]:
%%R
pd.13XN$Sample = rownames(pd.13XN)
pd.13XN = pd.13XN %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13XN.arr = arrange(pd.13XN, Replicate)
pd.13XN.arr

pd.13XN %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD     sd_PD
            (chr)    (dbl)     (dbl)
1             NTH 5.787702 0.5906482


In [295]:
%%R
pd.13XP$Sample = rownames(pd.13XP)
pd.13XP = pd.13XP %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

pd.13XP.arr = arrange(pd.13XP, Replicate)
pd.13XP.arr

pd.13XP %>% group_by(Land_Management) %>% summarise(mean_PD = mean(PD), sd_PD = sd(PD))


Source: local data frame [1 x 3]

  Land_Management  mean_PD    sd_PD
            (chr)    (dbl)    (dbl)
1             PTH 30.32662 2.125522


In [296]:
%%R
t.test(pd.13XN.arr$PD, pd.13XP.arr$PD, paired = TRUE)


	Paired t-test

data:  pd.13XN.arr$PD and pd.13XP.arr$PD
t = -20.855, df = 3, p-value = 0.0002411
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -28.28347 -20.79437
sample estimates:
mean of the differences 
              -24.53892 



### Pielou's

In [305]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13CN <- diversity(otu_tab.13CN )
## Species richness (S) and Pielou's evenness (J):
S.13CN <- specnumber(otu_tab.13CN ) ## rowSums(BCI > 0) does the same...
J.13CN <- H.13CN/log(S.13CN)

J.13CN <- J.13CN[c(1,3,4,2)]

J.13CN
print(mean(J.13CN))
print(sd(J.13CN))

[1] 0.6927556
[1] 0.1204529


In [308]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness
H.13CP <- diversity(otu_tab.13CP)
## Species richness (S) and Pielou's evenness (J):
S.13CP <- specnumber(otu_tab.13CP) ## rowSums(BCI > 0) does the same...
J.13CP <- H.13CP/log(S.13CP)

J.13CP <- J.13CP[c(4,3,1,2)]

J.13CP

print(mean(J.13CP))
print(sd(J.13CP))

[1] 0.8156578
[1] 0.02005656


In [309]:
%%R
t.test(J.13CN, J.13CP, paired = TRUE)


	Paired t-test

data:  J.13CN and J.13CP
t = -2.0089, df = 3, p-value = 0.1381
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.31759605  0.07179164
sample estimates:
mean of the differences 
             -0.1229022 



In [310]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13XN <- diversity(otu_tab.13XN )
## Species richness (S) and Pielou's evenness (J):
S.13XN <- specnumber(otu_tab.13XN ) ## rowSums(BCI > 0) does the same...
J.13XN <- H.13XN/log(S.13XN)

J.13XN <- J.13XN[c(1,3,4,2)]

J.13XN
print(mean(J.13XN))
print(sd(J.13XN))

[1] 0.8156668
[1] 0.02668541


In [311]:
%%R
#H - Shannons, S - Species Richness, J = Pielou's evenness

H.13XP <- diversity(otu_tab.13XP )
## Species richness (S) and Pielou's evenness (J):
S.13XP <- specnumber(otu_tab.13XP ) ## rowSums(BCI > 0) does the same...
J.13XP <- H.13XP/log(S.13XP)

J.13XP <- J.13XP[c(4,3,1,2)]

J.13XP
print(mean(J.13XP))
print(sd(J.13XP))

[1] 0.8175502
[1] 0.02106183


In [312]:
%%R
t.test(J.13XN, J.13XP, paired = TRUE)


	Paired t-test

data:  J.13XN and J.13XP
t = -0.12695, df = 3, p-value = 0.907
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.04909715  0.04533043
sample estimates:
mean of the differences 
           -0.001883364 



### Shannon

In [313]:
%%R
rich.13CN = estimate_richness(df.13C.N.phyl)

rich.13CP = estimate_richness(df.13C.P.phyl)

rich.13XN = estimate_richness(df.13X.N.phyl)

rich.13XP = estimate_richness(df.13X.P.phyl)

In [314]:
%%R
rich.13CN$Sample = rownames(rich.13CN)

rich.13CN = rich.13CN  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13CN %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             NTH          8.75     14.375      5.810355 16.84461     1.54455
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1      1.42606    0.6687666        3.065587    3.115406         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    3.774917 11.77126    7.037331 12.97939 0.6062096  0.1134999
  Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.04617217     0.4474421   1.19939       NA     1.290994


In [315]:
%%R
rich.13CP$Sample = rownames(rich.13CP)

rich.13CP = rich.13CP  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13CP %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             PTH        130.25   143.0891      6.932991 142.2834    5.759493
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1      3.97093    0.9656588          29.776    30.73193         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    6.849574 16.37716    5.938201 11.98646 0.2497478   0.103406
   Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.006339229      4.754484  3.597961       NA     1.290994


In [317]:
%%R
rich.13XN$Sample = rownames(rich.13XN)

rich.13XN = rich.13XN  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13XN %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             NTH         61.75      74.75      8.648176 73.04216    4.068777
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     3.357489    0.9355295        16.87298    18.97009         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    10.99621 14.10969    8.401311 10.09472 0.4971097  0.2484196
  Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.01874293      6.320788  2.992769       NA     1.290994


In [318]:
%%R
rich.13XP$Sample = rownames(rich.13XP)

rich.13XP = rich.13XP  %>% separate(Sample, c("Month", "Land_Management", "Replicate")) 

rich.13XP %>% group_by(Land_Management) %>% summarise_each(funs(mean, sd)) %>% as.data.frame


  Land_Management Observed_mean Chao1_mean se.chao1_mean ACE_mean se.ACE_mean
1             PTH        323.75   387.5382      18.11292 388.2508    9.179157
  Shannon_mean Simpson_mean InvSimpson_mean Fisher_mean Month_mean
1     4.722524    0.9765464        43.82602    96.98718         NA
  Replicate_mean Observed_sd Chao1_sd se.chao1_sd   ACE_sd se.ACE_sd Shannon_sd
1             NA    28.76775 11.33187    6.837812 15.63472 0.4516062  0.1174084
   Simpson_sd InvSimpson_sd Fisher_sd Month_sd Replicate_sd
1 0.004888301      7.610676  7.677888       NA     1.290994


In [319]:
%%R
rich.13CN.arr = rich.13CN %>% arrange(Replicate)
rich.13CP.arr = rich.13CP %>% arrange(Replicate)

In [320]:
%%R
t.test(rich.13CN.arr$Shannon, rich.13CP.arr$Shannon, paired = TRUE)


	Paired t-test

data:  rich.13CN.arr$Shannon and rich.13CP.arr$Shannon
t = -27.406, df = 3, p-value = 0.0001066
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -2.840388 -2.249352
sample estimates:
mean of the differences 
               -2.54487 



In [321]:
%%R
rich.13XN.arr = rich.13XN %>% arrange(Replicate)
rich.13XP.arr = rich.13XP %>% arrange(Replicate)

In [322]:
%%R
t.test(rich.13XN.arr$Shannon, rich.13XP.arr$Shannon, paired = TRUE)


	Paired t-test

data:  rich.13XN.arr$Shannon and rich.13XP.arr$Shannon
t = -10.23, df = 3, p-value = 0.001991
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.7896840 -0.9403859
sample estimates:
mean of the differences 
              -1.365035 

